In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210424.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,20/04/2021 17:48:06,Breathless (1960),The Umbrellas of Cherbourg (1964),The Red Shoes (1948),[italian hand gestures],GONE IN 90 MINUTES,[Default to Nicolas Cage],NaN
1,20/04/2021 17:52:18,The Red Shoes (1948),NaN,NaN,GONE IN 90 MINUTES,[italian hand gestures],[Default to Nicolas Cage],DEVIL'S ADVOCATE
2,20/04/2021 18:42:07,The Umbrellas of Cherbourg (1964),Breathless (1960),The Red Shoes (1948),[italian hand gestures],GONE IN 90 MINUTES,DEVIL'S ADVOCATE,[Default to Nicolas Cage]
3,21/04/2021 05:27:13,Breathless (1960),The Red Shoes (1948),The Umbrellas of Cherbourg (1964),DEVIL'S ADVOCATE,GONE IN 90 MINUTES,[italian hand gestures],[Default to Nicolas Cage]
4,21/04/2021 17:59:19,The Umbrellas of Cherbourg (1964),The Red Shoes (1948),Breathless (1960),[italian hand gestures],[Default to Nicolas Cage],GONE IN 90 MINUTES,DEVIL'S ADVOCATE
5,22/04/2021 07:25:04,Breathless (1960),The Red Shoes (1948),The Umbrellas of Cherbourg (1964),GONE IN 90 MINUTES,DEVIL'S ADVOCATE,[italian hand gestures],[Default to Nicolas Cage]
6,22/04/2021 08:13:22,The Umbrellas of Cherbourg (1964),The Red Shoes (1948),Breathless (1960),DEVIL'S ADVOCATE,[italian hand gestures],GONE IN 90 MINUTES,[Default to Nicolas Cage]
7,22/04/2021 09:23:23,Breathless (1960),The Umbrellas of Cherbourg (1964),The Red Shoes (1948),DEVIL'S ADVOCATE,[Default to Nicolas Cage],[italian hand gestures],GONE IN 90 MINUTES


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Breathless (1960)', 'The Umbrellas of Cherbourg (1964)',
       'The Red Shoes (1948)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Breathless (1960)': <Candidate('Breathless (1960)')>,
 'The Umbrellas of Cherbourg (1964)': <Candidate('The Umbrellas of Cherbourg (1964)')>,
 'The Red Shoes (1948)': <Candidate('The Red Shoes (1948)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Breathless (1960), The Umbrellas of Cherbourg (1964), The Red Shoes (1948))>,
 <Ballot(The Red Shoes (1948))>,
 <Ballot(The Umbrellas of Cherbourg (1964), Breathless (1960), The Red Shoes (1948))>,
 <Ballot(Breathless (1960), The Red Shoes (1948), The Umbrellas of Cherbourg (1964))>,
 <Ballot(The Umbrellas of Cherbourg (1964), The Red Shoes (1948), Breathless (1960))>,
 <Ballot(Breathless (1960), The Red Shoes (1948), The Umbrellas of Cherbourg (1964))>,
 <Ballot(The Umbrellas of Cherbourg (1964), The Red Shoes (1948), Breathless (1960))>,
 <Ballot(Breathless (1960), The Umbrellas of Cherbourg (1964), The Red Shoes (1948))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                            Votes  Status
---------------------------------  -------  --------
Breathless (1960)                        4  Elected
The Umbrellas of Cherbourg (1964)        3  Rejected
The Red Shoes (1948)                     1  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
[italian hand gestures]          3  Hopeful
DEVIL'S ADVOCATE                 3  Hopeful
GONE IN 90 MINUTES               2  Rejected
[Default to Nicolas Cage]        0  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
[italian hand gestures]          4  Elected
DEVIL'S ADVOCATE                 4  Rejected
GONE IN 90 MINUTES               0  Rejected
[Default to Nicolas Cage]        0  Rejected

